In [32]:
import pandas as pd
import numpy as np
import requests
import json
import os
import re

data_folder = 'data/'

In [33]:
API_KEY = ""

with open('api_key.txt') as f:
    API_KEY = f.read()

HEADERS = {"X-Api-Key": API_KEY}

In [3]:

def create_folder(path):
    if not os.path.exists(path):
        os.makedirs(path)

create_folder(data_folder)
create_folder(data_folder + 'commities')
create_folder(data_folder + 'senate_members')
create_folder(data_folder + 'votes')
create_folder(data_folder + 'lobby')

In [4]:
# Request config
req_str = "https://api.propublica.org/congress/v1/senate/votes/recent.json?offset={offset}"

res = requests.get(req_str.format(offset=460), headers = HEADERS)

print(json.dumps(res.json(), indent=4))

{
    "status": "OK",
    "copyright": "Copyright (c) 2018 Pro Publica Inc. All Rights Reserved.",
    "results": {
        "chamber": "Senate",
        "offset": 460,
        "num_results": 20,
        "votes": [
            {
                "congress": 115,
                "chamber": "Senate",
                "session": 1,
                "roll_call": 84,
                "source": "https://www.senate.gov/legislative/LIS/roll_call_votes/vote1151/vote_115_1_00084.xml",
                "url": "https://www.senate.gov/legislative/LIS/roll_call_lists/roll_call_vote_cfm.cfm?congress=115&session=1&vote=00084",
                "vote_uri": "https://api.propublica.org/congress/v1/115/senate/sessions/1/votes/84.json",
                "bill": {
                    "bill_id": "hjres57-115",
                    "number": "H.J.RES.57",
                    "sponsor_id": "R000592",
                    "api_uri": "https://api.propublica.org/congress/v1/115/bills/hjres57.json",
                    "tit

# Fetching senate members (congress 80 to 115)

In [5]:
u = "https://api.propublica.org/congress/v1/{congress}/senate/members.json"

for i in range(80, 115 + 1):
    results = requests.get(url = u.format(congress=i), headers=HEADERS)
    df = pd.io.json.json_normalize(results.json()['results'][0]['members'])
    df.to_csv("data/senate_members/senate_members_{congress}.csv".format(congress=i))

In [6]:
df.head()

,api_uri,contact_form,crp_id,cspan_id,date_of_birth,dw_nominate,facebook_account,fax,fec_candidate_id,first_name,...,state_rank,suffix,title,total_present,total_votes,twitter_account,url,votes_with_party_pct,votesmart_id,youtube_account
0,https://api.propublica.org/congress/v1/members...,http://www.alexander.senate.gov/public/index.c...,N00009888,5,1940-07-03,0.323,senatorlamaralexander,202-228-3398,S2TN00058,Lamar,...,senior,None,"Senator, 2nd Class",0,544,SenAlexander,https://www.alexander.senate.gov/public,96.91,15691,lamaralexander
1,https://api.propublica.org/congress/v1/members...,https://www.baldwin.senate.gov/feedback,N00004367,57884,1962-02-11,-0.523,TammyBaldwin,202-225-6942,S2WI00219,Tammy,...,junior,None,"Senator, 1st Class",1,544,SenatorBaldwin,https://www.baldwin.senate.gov,94.09,3470,witammybaldwin
2,https://api.propublica.org/congress/v1/members...,https://www.barrasso.senate.gov/public/index.c...,N00006236,1024777,1952-07-21,0.532,johnbarrasso,202-224-1724,S6WY00068,John,...,junior,None,"Senator, 1st Class",0,544,SenJohnBarrasso,https://www.barrasso.senate.gov,96.69,52662,barrassowyo
3,https://api.propublica.org/congress/v1/members...,https://www.bennet.senate.gov/?p=contact,N00030608,1031622,1964-11-28,-0.206,senbennetco,202-228-5097,S0CO00211,Michael,...,senior,None,"Senator, 3rd Class",1,544,SenBennetCo,https://www.bennet.senate.gov,90.17,110942,SenatorBennet
4,https://api.propublica.org/congress/v1/members...,https://www.blumenthal.senate.gov/contact/,N00031685,21799,1946-02-13,-0.425,SenBlumenthal,202-224-9673,S0CT00177,Richard,...,senior,None,"Senator, 3rd Class",1,544,SenBlumenthal,https://www.blumenthal.senate.gov,90.39,1568,SenatorBlumenthal


# Fetching vote data from senate

In [7]:
# Request config
u = "https://api.propublica.org/congress/v1/senate/votes/recent.json?offset={offset}"

votes_list = []
data_available = True
request_offset = 0

while data_available :
    res = requests.get(url = u.format(offset = request_offset), headers = HEADERS)
    if('results' in res.json()):
        data_available = int(res.json()['results']['num_results']) > 0

        if data_available:
            votes = res.json()['results']['votes']
            votes_list.append(pd.io.json.json_normalize(votes, record_prefix=True))
    else:
        print(str(request_offset) + ' - Error: ' + res.json()['error'])
    request_offset += 20

1040 - Error: Internal Server Error
1080 - Error: Internal Server Error
2280 - Error: Internal Server Error
2680 - Error: Internal Server Error


In [8]:
df = pd.concat(votes_list,sort=True)
df.to_csv(data_folder + "votes/votes.csv")

In [9]:
df.head()

,amendment.api_uri,amendment.number,amendment.sponsor,amendment.sponsor_id,amendment.sponsor_party,amendment.sponsor_state,amendment.sponsor_uri,bill.api_uri,bill.bill_id,bill.latest_action,...,tie_breaker,tie_breaker_vote,time,total.no,total.not_voting,total.present,total.yes,url,vote_type,vote_uri
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://api.propublica.org/congress/v1/115/bil...,hr302-115,Motion by Senator McConnell to refer to Senate...,...,,,17:31:00,7,3,0,90,https://www.senate.gov/legislative/LIS/roll_ca...,3/5,https://api.propublica.org/congress/v1/115/sen...
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,,,12:41:00,1,1,0,98,https://www.senate.gov/legislative/LIS/roll_ca...,1/2,https://api.propublica.org/congress/v1/115/sen...
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,,,13:46:00,49,0,0,51,https://www.senate.gov/legislative/LIS/roll_ca...,1/2,https://api.propublica.org/congress/v1/115/sen...
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,,,14:45:00,49,1,0,50,https://www.senate.gov/legislative/LIS/roll_ca...,1/2,https://api.propublica.org/congress/v1/115/sen...
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,,,14:16:00,19,1,0,80,https://www.senate.gov/legislative/LIS/roll_ca...,1/2,https://api.propublica.org/congress/v1/115/sen...


# Fetching Commities from senate

In [10]:
# Request config
u = "https://api.propublica.org/congress/v1/{congress}/senate/committees.json"

for i in range(110, 115 + 1):
    results = requests.get(url = u.format(congress=i), headers=HEADERS)
    df = pd.io.json.json_normalize(results.json()['results'][0]['committees'])
    df.to_csv("data/commities/commities_{congress}.csv".format(congress=i))

In [11]:
df.head()

,api_uri,chair,chair_id,chair_party,chair_state,chair_uri,chamber,id,name,ranking_member_id,subcommittees,url
0,https://api.propublica.org/congress/v1/115/sen...,Charles E. Grassley,G000386,R,IA,https://api.propublica.org/congress/v1/members...,Senate,SCNC,Caucus on International Narcotics Control,F000062,[],http://www.drugcaucus.senate.gov/
1,https://api.propublica.org/congress/v1/115/sen...,Pat Roberts,R000307,R,KS,https://api.propublica.org/congress/v1/members...,Senate,SSAF,"Committee on Agriculture, Nutrition, and Forestry",S000770,"[{'id': 'SSAF13', 'name': 'Subcommittee on Com...",https://www.agriculture.senate.gov/
2,https://api.propublica.org/congress/v1/115/sen...,Thad Cochran,C000567,R,MS,https://api.propublica.org/congress/v1/members...,Senate,SSAP,Committee on Appropriations,L000174,"[{'id': 'SSAP18', 'name': 'Departments of Labo...",https://www.appropriations.senate.gov/
3,https://api.propublica.org/congress/v1/115/sen...,John McCain,M000303,R,AZ,https://api.propublica.org/congress/v1/members...,Senate,SSAS,Committee on Armed Services,R000122,"[{'id': 'SSAS16', 'name': 'Strategic Forces Su...",https://www.armed-services.senate.gov/
4,https://api.propublica.org/congress/v1/115/sen...,Michael D. Crapo,C000880,R,ID,https://api.propublica.org/congress/v1/members...,Senate,SSBK,"Committee on Banking, Housing, and Urban Affairs",B000944,"[{'id': 'SSBK04', 'name': 'Securities, Insuran...",https://www.banking.senate.gov/public/


# Fetching Lobbying from senate

In [12]:
re.compile("\((.*)\"([\w ]+)\"(.*)\)")

re.compile(r'\((.*)"([\w ]+)"(.*)\)', re.UNICODE)

In [13]:
# Request config
u = "https://api.propublica.org/congress/v1/lobbying/latest.json?offset={offset}"

votes_list = []
data_available = True
request_offset = 0

while data_available :
    res = requests.get(url = u.format(offset = request_offset), headers = HEADERS)
    j  = res.text.replace("\\\"","").replace("\\","").replace("\")", ")").replace("(\"", "(")

    try:
        res = json.loads(j)       
        if('results' in res):
            #print(request_offset)
            data_available = int(res['results'][0]['num_results']) > 0

            if data_available:
                votes = res['results'][0]['lobbying_representations']
                votes_list.append(pd.io.json.json_normalize(votes, record_prefix=True))
        else:
            print(str(request_offset) + ' - Error: ' + res['error'])
    except:
        print(str(request_offset) + ' - Error: Json File badly encoded')
    request_offset += 20

1120 - Error: Json File badly encoded
2820 - Error: Json File badly encoded
2900 - Error: Json File badly encoded
3540 - Error: Json File badly encoded
3840 - Error: Json File badly encoded
4120 - Error: Json File badly encoded
5120 - Error: Json File badly encoded
5240 - Error: Json File badly encoded
5840 - Error: Json File badly encoded
5880 - Error: Json File badly encoded
7820 - Error: Json File badly encoded
8620 - Error: Json File badly encoded
8960 - Error: Json File badly encoded
9160 - Error: Json File badly encoded
11360 - Error: Json File badly encoded
11920 - Error: Json File badly encoded
12380 - Error: Json File badly encoded
12560 - Error: Json File badly encoded
14960 - Error: Json File badly encoded
15220 - Error: Json File badly encoded
15400 - Error: Json File badly encoded
15440 - Error: Json File badly encoded
15800 - Error: Json File badly encoded
16460 - Error: Json File badly encoded
17340 - Error: Json File badly encoded
17620 - Error: Json File badly encoded


In [14]:
df = pd.concat(votes_list,sort=True)
df.to_csv(data_folder + "lobby/lobby.csv")

In [16]:
df.head()

,effective_date,house_id,id,inhouse,lobbying_client.general_description,lobbying_client.name,lobbying_registrant.general_description,lobbying_registrant.name,lobbyists,report_type,report_year,senate_id,signed_date,specific_issues,xml_filename
0,,440230001,301017554,false,,Neema Ltd.,,TUSK VENTURES LLC,"[{'name': 'Robert Greenlee', 'covered_position...",,,,,[Issues surrounding U.S. acceptance of the Rep...,301017554.xml
1,,415950000,301001779,false,,Demotech,,Michael Stinziano,[],,,400720167-12,,[null],301001779.xml
2,,317190000,300985209,false,,AMERICAN BUSINESS CONFERENCE,,AMERICAN BUSINESS CONFERENCE,[],,,,,[null],300985209.xml
3,,355730006,300970730,false,,GULF PORTS ASSN,,Ms. Pat Younger,"[{'name': 'Pat Younger', 'covered_position': ''}]",,,,,[Appropriations for channel improvement projec...,300970730.xml
4,,320390021,300969852,false,,Newington-Cropsey Foundation Academy of Art,,The Chwat Group,"[{'name': 'john chwat', 'covered_position': ''...",,,,,[100th Anniversary of the National Park Service],300969852.xml


# Getting voting position by member

In [38]:
raw_senators = pd.DataFrame()

for i in range (115, 80 - 1, -1):
    df = pd.read_csv("data/senate_members/senate_members_{congress}.csv".format(congress = i))
    df['congress'] = i
    raw_senators = pd.concat([raw_senators, df], sort=False)
    
senators_id = raw_senators['id'].unique()

In [46]:
u = "https://api.propublica.org/congress/v1/members/{member_id}/votes.json?offset={offset}"

import os
import json

completed_ids = os.listdir("data/votes")

for senator_id in senators_id[2::4]:

    if "votes_{id}.csv".format(id=senator_id) in completed_ids:
        continue
        
    error_raised = False
    votes_list = []
    data_available = True
    request_offset = 0

    while data_available :
        res = requests.get(url = u.format(member_id = senator_id, offset = request_offset), headers = HEADERS)
        jObj = json.loads(res.text.replace('\n', ' '))
                
        #print(res.text)
        if(res.status_code == 200):
            data_available = int(jObj['results'][0]['num_results']) > 0

            if data_available:
                print(str(senator_id) + " offset: " + str(request_offset), end='\r')
                votes = jObj['results'][0]['votes']
                votes_list.append(pd.io.json.json_normalize(votes, record_prefix=True))
                
        else:
            print(str(res.status_code))
            error_raised = True
            break
        
        request_offset += 20
        
    if not error_raised and len(votes_list) > 0:
        df = pd.concat(votes_list,sort=False)
        df.to_csv(data_folder + "votes/votes_{id}.csv".format(id=senator_id), index=False)
